<a href="https://colab.research.google.com/github/georgesnape01/dissertation/blob/main/notebooks/languagemodels/diss_llm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Success: Machine Learning Models for CBT Outcomes

## Load Packages & Mount Drive

In [ ]:
from google.colab import drive
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

drive.mount('/content/drive')
notebook_path = '/content/drive/MyDrive/Colab Notebooks/'
sys.path.append(notebook_path)
import cleaning_functions as cleanf
import eda_functions as edaf
import preparation_functions as prepf
import modelling_functions as modelf

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preparation

In [ ]:
# # create different processed datasets
# data, raw_df = cleanf.load_data()
# data = cleanf.Clean_Data(data)

# prep1_df = prepf.Prepare_Data(
#     df=data,
#     quasi_thresh=0.999,
#     corr_thresh=0.999,
#     load_matrices=True,
#     col_thresh=0.9,
#     row_thresh=0.9,
#     scale_method='normalise',
#     imputation_method='iterative',
#     #n_neighbours=3,
#     max_iter=10,
#     k_features=200)

** **Convert targets to integer, fix imputed values** **

In [ ]:
# cleanf.manage_data(prep1_df, 'prep1_df', 'save')
# temp = cleanf.manage_data(None, 'prep1_df', 'load')

In [ ]:
prep7_df = cleanf.manage_data(None, 'prep7_df', 'load')

# New

In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 

In [ ]:
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
def serialise_data(df, target):
    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
    explanatory_df = df.drop(['ReliableChangeDesc', 'ReliableRecovery', 'Recovery'] + EndDesc_cols, axis = 1)
    text = explanatory_df.apply(lambda row: row.to_json(), axis=1)
    text_df = pd.DataFrame({
        'text': text,
        'label': df[target]})
    text_df = text_df.dropna()

    return text_df

In [ ]:
text_df = serialise_data(prep7_df, 'Recovery')
train_df, test_df = train_test_split(text_df, test_size=0.10)

# define hyperparameter
train_args ={"reprocess_input_data": True,
             'learning_rate': 1e-5,
             "fp16":False,
             "num_train_epochs": 4,
             "overwrite_output_dir": True} # avoid existing outpit directory error

# llm
model = ClassificationModel(
    "bert", "distilbert-base-cased",
    num_labels=2,
    use_cuda=False,
    args=train_args)

model.train_model(train_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.wei

0it [00:00, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/44 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/44 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/44 [00:00<?, ?it/s]

Running Epoch 4 of 4:   0%|          | 0/44 [00:00<?, ?it/s]

(176, 0.7158142172477462)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=accuracy_score)
result

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

{'mcc': 0.0,
 'accuracy': 0.55,
 'f1_score': 0.3548387096774194,
 'tp': 0,
 'tn': 22,
 'fp': 0,
 'fn': 18,
 'auroc': 0.5656565656565656,
 'auprc': 0.48837327337629266,
 'acc': 0.55,
 'eval_loss': 0.6879599690437317}

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from sklearn.metrics import accuracy_score
from simpletransformers.classification import ClassificationModel

In [ ]:
def XGBoost_HyperparameterTuning(df, param_grid, outer_k=5, inner_k=5):

    # dataset
    X = df['text']
    y = df['label']

    # initialise lists
    scores, preds, actuals = [], [], []

    # cross validation and hyperparameter tuning
    outer_cv = StratifiedKFold(n_splits = outer_k, shuffle = True)
    inner_cv = StratifiedKFold(n_splits = inner_k, shuffle = True) # both set to k for now
    for train_index, test_index in outer_cv.split(X, y):

        # outer CV train and test sets
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        train_df = pd.DataFrame({'text': X_train, 'label': y_train})
        test_df = pd.DataFrame({'text': X_test, 'label': y_test})

        model = ClassificationModel(
                    "bert", "distilbert-base-cased",
                    num_labels=2,
                    use_cuda=False,
                    args=params)

        # hyper-parameter tuning
        grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=inner_cv, scoring='accuracy', verbose=0, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        print("Inner CV accuracy: {}".format(grid_search.best_score_)) # validation sets

        # optimal model
        estimator = grid_search.best_estimator_

        # count features selected
        support = estimator.named_steps['FS'].get_support()
        num_feat = np.sum(support)
        print("Number of selected features {0}".format(num_feat))

        # features selected
        col_index = np.where(support)[0]
        col_names = [cols[col] for col in col_index]
        print("Selected features {0}".format(col_names))

        # hyperparameters selected
        print("Max depth {0}".format(estimator.named_steps["classifier"].max_depth))

        # evaluating optimised model on test
        predictions = estimator.predict(X_test)
        score = metrics.accuracy_score(y_test, predictions)
        scores.append(score)
        print('Outer CV accuracy: {}'.format(score)) # test sets

        print("--------------------------------------------------")

        probs = estimator.predict_proba(X_test)[:, 1]
        preds.extend(probs)
        actuals.extend(y_test)

    return scores, preds, actuals

In [ ]:
def BERT_HyperparameterTuning(df, param_grid, outer_k=5, inner_k=5):

    # dataset
    X = df['text']
    y = df['label']

    # initialise lists
    scores, preds, actuals = [], [], []

    # cross validation and hyperparameter tuning
    outer_cv = StratifiedKFold(n_splits=outer_k, shuffle=True)
    inner_cv = StratifiedKFold(n_splits=inner_k, shuffle=True)  # both set to k for now

    for train_index, test_index in outer_cv.split(X, y):
        # outer CV train and test sets
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        train_df = pd.DataFrame({'text': X_train, 'label': y_train})
        test_df = pd.DataFrame({'text': X_test, 'label': y_test})

        best_score = 0
        best_params = None

        for params in ParameterGrid(param_grid):
            fold_scores = []

            for inner_train_index, inner_val_index in inner_cv.split(X_train, y_train):
                X_inner_train, X_val = X_train.iloc[inner_train_index], X_train.iloc[inner_val_index]
                y_inner_train, y_val = y_train.iloc[inner_train_index], y_train.iloc[inner_val_index]

                inner_train_df = pd.DataFrame({'text': X_inner_train, 'label': y_inner_train})
                val_df = pd.DataFrame({'text': X_val, 'label': y_val})

                # Train BERT model with current params
                model = ClassificationModel(
                    "bert", "distilbert-base-cased",
                    num_labels=2,
                    use_cuda=False,
                    args=params
                )

                model.train_model(inner_train_df)

                # Evaluate on validation set
                result, model_outputs, wrong_predictions = model.eval_model(val_df, acc=accuracy_score)
                fold_scores.append(result['acc'])

            mean_score = sum(fold_scores) / len(fold_scores)
            print(f'Inner CV accuracy: {mean_score}')
            print("--------------------------------------------------")

            if mean_score > best_score:
                best_score = mean_score
                best_params = params

        # Train final model with best hyperparameters on the full training set
        model = ClassificationModel(
            "bert", "distilbert-base-cased",
            num_labels=2,
            use_cuda=False,
            args=best_params
        )
        model.train_model(train_df)

        # Evaluate on the outer test set
        result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=accuracy_score)
        scores.append(result['acc'])
        preds.extend(model_outputs)
        actuals.extend(test_df['label'])

        print(f'External CV accuracy: {result}')
        print("--------------------------------------------------")

    print(f'Final Average Accuracy: {sum(scores) / len(scores):.4f}')

    # Return or print results as needed
    return scores, preds, actuals


In [ ]:
# Example usage:
param_grid = {
    'learning_rate': [1e-5, 3e-5],
    'num_train_epochs': [2, 3],
    'fp16': [False],
    'reprocess_input_data': [True],
    'overwrite_output_dir': [True],
    'train_batch_size': [16, 32]}

# Assuming `prep7_df` is your initial DataFrame and 'Recovery' is the label column
text_df = serialise_data(prep7_df, 'Recovery')
scores, preds, actuals = BERT_HyperparameterTuning(text_df, param_grid)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.wei

0it [00:00, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 2 of 2:   0%|          | 0/16 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/8 [00:00<?, ?it/s]